## Pandasでよく使う操作

In [1]:
import numpy as np
import pandas as pd

### 辞書としてのSeries
pythonの辞書よりも遥かに効率的である。

順番を指定する場合

In [2]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

順番を指定しない場合

In [3]:
population_dict = {'California': 38332521,
                   'Texas': 26448193,
                   'New York': 19651127,
                   'Florida': 19552860,
                   'Illinois': 12882135}
population = pd.Series(population_dict)
population

California    38332521
Texas         26448193
New York      19651127
Florida       19552860
Illinois      12882135
dtype: int64

In [4]:
population['California']

38332521

In [5]:
#スライス機能も使える
population['California':'Illinois'] #numpyのスライス違ってスライスの最後は含まれる

California    38332521
Texas         26448193
New York      19651127
Florida       19552860
Illinois      12882135
dtype: int64

当然python辞書と同じような事もできる

In [6]:
population.keys()

Index(['California', 'Texas', 'New York', 'Florida', 'Illinois'], dtype='object')

In [7]:
print("New York" in population)
print(38332521 in population)

True
False


In [8]:
list(population.items())

[('California', 38332521),
 ('Texas', 26448193),
 ('New York', 19651127),
 ('Florida', 19552860),
 ('Illinois', 12882135)]

### 順序付き集合としてのIndex
seriesもdataframeもIndexを持つが、順序付き集合としての機能を持つ。いちいちpython setに変換せずに済むので便利

In [9]:
indA = pd.Index([1, 3, 5, 7, 9])
indB = pd.Index([2, 3, 5, 7, 11])
print(indA & indB) #積集合
print(indA | indB) #和集合
print(indA ^ indB) #対称差

Int64Index([3, 5, 7], dtype='int64')
Int64Index([1, 2, 3, 5, 7, 9, 11], dtype='int64')
Int64Index([1, 2, 9, 11], dtype='int64')


### DataFrameの作り方

ネットでよくやられてるやつ でも列の順番が入れ替わったりする

In [10]:
pd.DataFrame({
    "first":[5,2],
    "second":[3,4]
})

,first,second
0,5,3
1,2,4


辞書のリストから作る　前者よりは賢そう

In [11]:
data = [{'a': i, 'b': 2 * i}
        for i in range(3)]
data

[{'a': 0, 'b': 0}, {'a': 1, 'b': 2}, {'a': 2, 'b': 4}]

In [12]:
pd.DataFrame(data)

,a,b
0,0,0
1,1,2
2,2,4


Numpy配列から作る deepの結果をpandasに変換するときとか便利そう

In [13]:
data = np.random.rand(3, 2)
print(data)
pd.DataFrame(data,
             columns=['foo', 'bar'],
             index=['a', 'b', 'c'])

[[0.12744384 0.71567935]
 [0.93381723 0.99381034]
 [0.92376241 0.86336136]]


,foo,bar
a,0.127444,0.715679
b,0.933817,0.993810
c,0.923762,0.863361


### データフレームに新しい行を作る

すでにあるデータから計算する場合

In [14]:
area = pd.Series({'California': 423967, 'Texas': 695662,
                  'New York': 141297, 'Florida': 170312,
                  'Illinois': 149995})
pop = pd.Series({'California': 38332521, 'Texas': 26448193,
                 'New York': 19651127, 'Florida': 19552860,
                 'Illinois': 12882135})
data = pd.DataFrame({'area':area, 'pop':pop})
data

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


In [15]:
data['density'] = data['pop'] / data['area'] #なんと簡単
data

,area,pop,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


### 条件に合うデータに絞る
locはmaskとしても働く

In [16]:
data.density>100

California    False
Texas         False
New York       True
Florida        True
Illinois      False
Name: density, dtype: bool

In [17]:
data.loc[data.density>100]

,area,pop,density
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121


In [18]:
data.loc[data.density>100,["area", "pop"]] #行も一緒に指定できる

,area,pop
New York,141297,19651127
Florida,170312,19552860


### Pandasのデータに同じ加工をする
実はnumpyをにそのまま噛ませれば良い

In [19]:
np.sqrt(data) #返り値もpandasである

,area,pop,density
California,651.127484,6191.326271,9.508624
Texas,834.063547,5142.780668,6.165934
New York,375.894932,4432.959170,11.793080
Florida,412.688745,4421.861599,10.714762
Illinois,387.291880,3589.169124,9.267349


### 欠損値の扱い

null値の検出

In [20]:
data = pd.Series([1, np.nan, 'hello', None])
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [21]:
data.isnull(), data.notnull()

(0    False
 1     True
 2    False
 3     True
 dtype: bool, 0     True
 1    False
 2     True
 3    False
 dtype: bool)

欠損値の除外

In [22]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [23]:
df.dropna()

,0,1,2
1,2.0,3.0,5


オプションとして,howとaxisとthreshがある。`how="all"`ですべてnullの行(列)を削除。デフォでは`how="any"`。axisは、軸の指定。またthreshでnull以外がいくつ以上で残すか決めることもできる

In [24]:
df.dropna(axis='columns')

,2
0,2
1,5
2,6


In [25]:
df.dropna(axis="rows", thresh=3) #非nullが3つ以上なのは1だけ

,0,1,2
1,2.0,3.0,5


欠損値の埋め合わせ

In [26]:
df #データはこんな感じ

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


0で埋め合わせる

In [27]:
df.fillna(0) #他の数字で埋め合わせることも可能

,0,1,2
0,1.0,0.0,2
1,2.0,3.0,5
2,0.0,4.0,6


前の数字で埋め合わせる

In [28]:
df.fillna(method="ffill") #後ろ向きのbfillもある

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,2.0,4.0,6


In [29]:
df.fillna(method="bfill", axis="columns")

,0,1,2
0,1.0,2.0,2.0
1,2.0,3.0,5.0
2,4.0,4.0,6.0


### 階層型インデックス
今までpanelとかで操作してきたけど多分こっちのほうがめっちゃ便利

例えばこんなもの

In [30]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
index = pd.MultiIndex.from_tuples(index)
index #まじでインデックスだけを持っている

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [31]:
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations,index=index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [32]:
pop.loc["New York":"Texas", 2000] #複数のindexで絞ることも可能

New York  2000    18976457
Texas     2000    20851820
dtype: int64

Seriesならばunstackとstackでdfに変換したりできる

In [33]:
pop.unstack()

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [34]:
pop.unstack().stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

dfでの計算操作も今まで通り

In [35]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [36]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


MultiIndexの作り方

DataFrame作成時に暗黙的に作成するやり方。→indexにリストのリストを入れれば良し

In [37]:
np.arange(8).reshape(4,2)

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7]])

In [38]:
df = pd.DataFrame(np.arange(8).reshape(4, 2),
                  index=[['a', 'a', 'b', 'b'], 
                         [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1  data2
a 1      0      1
  2      2      3
b 1      4      5
  2      6      7

辞書からも暗黙的に作れる。もうちょっとスマートである。ポイントは、keyにタプルを作ることである。

In [39]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

明示的にMultiIndexを作る
様々な作り方がある

In [40]:
#配列から作る
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], 
                           [1, 2, 1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [41]:
#タプルから作る
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [42]:
#デカルト積から作る
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

複数のindexに名前をふる

In [43]:
pop.index.names = ['state', 'year'] #リストで複数指定可能

In [44]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

データフレームの行列両方にマルチインデクスにする

In [45]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), decimals=1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
# DFを作るときに
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      40.0  37.7  42.0  36.2  33.0  37.7
     2      18.0  35.7  40.0  38.5  39.0  37.3
2014 1      21.0  38.2  44.0  38.2  30.0  39.8
     2      30.0  36.2  28.0  37.7  42.0  37.3

データの指定

In [46]:
health_data["Bob","HR"]

year  visit
2013  1        40.0
      2        18.0
2014  1        21.0
      2        30.0
Name: (Bob, HR), dtype: float64

In [47]:
health_data.loc[:,"Bob":"Guido"]

subject      Bob       Guido      
type          HR  Temp    HR  Temp
year visit                        
2013 1      40.0  37.7  42.0  36.2
     2      18.0  35.7  40.0  38.5
2014 1      21.0  38.2  44.0  38.2
     2      30.0  36.2  28.0  37.7

In [48]:
health_data.loc[:,("Bob","Temp"):("Guido","HR")]

subject      Bob Guido
type        Temp    HR
year visit            
2013 1      37.7  42.0
     2      35.7  40.0
2014 1      38.2  44.0
     2      36.2  28.0

In [49]:
idx = pd.IndexSlice #idxスライスを使わないとエラーになる(pythonスライスはだめ)
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,40.0,42.0,33.0
2014,1,21.0,44.0,30.0


ソートする必要がある場合もある

In [50]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.811727
      2      0.715130
c     1      0.497778
      2      0.599727
b     1      0.160833
      2      0.797465
dtype: float64

In [51]:
#data.loc["a":"b"] 実行すると以下のようなエラーになります

# ---------------------------------------------------------------------------
# UnsortedIndexError                        Traceback (most recent call last)
# <ipython-input-103-2a5d331a3674> in <module>
# ----> 1 data.loc["a":"b"]
#中略
#UnsortedIndexError: 'Key length (1) was greater than MultiIndex lexsort depth (0)'

In [52]:
data = data.sort_index() #ソートしてやればエラーは出なくなる
print(data)
data.loc["a":"b"]

char  int
a     1      0.811727
      2      0.715130
b     1      0.160833
      2      0.797465
c     1      0.497778
      2      0.599727
dtype: float64


char  int
a     1      0.811727
      2      0.715130
b     1      0.160833
      2      0.797465
dtype: float64

すでにあるdfからMultiIndex化する。
set_indexで複数の列を指定すればok。便利なので多用しそう。

In [53]:
pop_flat = pop.reset_index(name="population")
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [54]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

MultiIndexのデータの集約
levelを指定してやれば良い

In [55]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      40.0  37.7  42.0  36.2  33.0  37.7
     2      18.0  35.7  40.0  38.5  39.0  37.3
2014 1      21.0  38.2  44.0  38.2  30.0  39.8
     2      30.0  36.2  28.0  37.7  42.0  37.3

In [56]:
health_data.mean(level='year') #年より下のレベル visitが潰されて集約されている

subject   Bob       Guido          Sue       
type       HR  Temp    HR   Temp    HR   Temp
year                                         
2013     29.0  36.7  41.0  37.35  36.0  37.50
2014     25.5  37.2  36.0  37.95  36.0  38.55

MultiIndexとしてデータフレームを結合する
pd.concatにkeysを指定すれば良い

In [57]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])

pd.concat([df1,df2])

,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


In [58]:
pd.concat([df1,df2],keys=["x","y"])

A   B
x 1  A1  B1
  2  A2  B2
y 3  A3  B3
  4  A4  B4

### 集約とグループ化
代表的な統計量の計算などをすばやくやる

In [59]:
import seaborn as sns
planets = sns.load_dataset('planets')
print(planets.shape)
print(planets.isna().sum()) #欠損値がある
planets.head()

(1035, 6)
method              0
number              0
orbital_period     43
mass              522
distance          227
year                0
dtype: int64


,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


すべての特徴の概要

In [60]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


グループごとに情報を統計量に集約する
groupbyを使う

In [61]:
planets.groupby("method").describe()

distance                                     \
                                 count         mean          std      min   
method                                                                      
Astrometry                         2.0    17.875000     4.094148    14.98   
Eclipse Timing Variations          4.0   315.360000   213.203907   130.72   
Imaging                           32.0    67.715937    53.736817     7.69   
Microlensing                      10.0  4144.000000  2076.611556  1760.00   
Orbital Brightness Modulation      2.0  1180.000000     0.000000  1180.00   
Pulsar Timing                      1.0  1200.000000          NaN  1200.00   
Pulsation Timing Variations        0.0          NaN          NaN      NaN   
Radial Velocity                  530.0    51.600208    45.559381     1.35   
Transit                          224.0   599.298080   913.876990    38.00   
Transit Timing Variations          3.0  1104.333333   915.819487   339.00   

                                                                         mass  \
                                     25%       50%        75%      max  count   
method                                                                          
Astrometry                       16.4275    17.875    19.3225    20.77    0.0   
Eclipse Timing Variations       130.7200   315.360   500.0000   500.00    2.0   
Imaging                          22.1450    40.395   132.6975   165.00    0.0   
Microlensing                   2627.5000  3840.000  4747.5000  7720.00    0.0   
Orbital Brightness Modulation  1180.0000  1180.000  1180.0000  1180.00    0.0   
Pulsar Timing                  1200.0000  1200.000  1200.0000  1200.00    0.0   
Pulsation Timing Variations          NaN       NaN        NaN      NaN    0.0   
Radial Velocity                  24.4125    40.445    59.2175   354.00  510.0   
Transit                         200.0000   341.000   650.0000  8500.00    1.0   
Transit Timing Variations       597.0000   855.000  1487.0000  2119.00    0.0   

                                          ...   orbital_period                 \
                                   mean   ...              75%            max   
method                                    ...                                   
Astrometry                          NaN   ...       823.590000    1016.000000   
Eclipse Timing Variations      5.125000   ...      5767.000000   10220.000000   
Imaging                             NaN   ...     94250.000000  730000.000000   
Microlensing                        NaN   ...      3550.000000    5100.000000   
Orbital Brightness Modulation       NaN   ...         0.943908       1.544929   
Pulsar Timing                       NaN   ...        98.211400   36525.000000   
Pulsation Timing Variations         NaN   ...      1170.000000    1170.000000   
Radial Velocity                2.630699   ...       982.000000   17337.500000   
Transit                        1.470000   ...        16.145700     331.600590   
Transit Timing Variations           NaN   ...       108.505500     160.000000   

                                year                                          \
                               count         mean       std     min      25%   
method                                                                         
Astrometry                       2.0  2011.500000  2.121320  2010.0  2010.75   
Eclipse Timing Variations        9.0  2010.000000  1.414214  2008.0  2009.00   
Imaging                         38.0  2009.131579  2.781901  2004.0  2008.00   
Microlensing                    23.0  2009.782609  2.859697  2004.0  2008.00   
Orbital Brightness Modulation    3.0  2011.666667  1.154701  2011.0  2011.00   
Pulsar Timing                    5.0  1998.400000  8.384510  1992.0  1992.00   
Pulsation Timing Variations      1.0  2007.000000       NaN  2007.0  2007.00   
Radial Velocity                553.0  2007.518987  4.249052  1989.0  2005.00   
Transit                        397.0  20

In [62]:
planets.groupby("method")["orbital_period"].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

In [63]:
planets.groupby("method").aggregate(["min", np.median, max]) #任意の統計量を計算できる #aggreate()

number            orbital_period                \
                                 min median max            min        median   
method                                                                         
Astrometry                         1      1   1     246.360000    631.180000   
Eclipse Timing Variations          1      2   2    1916.250000   4343.500000   
Imaging                            1      1   4    4639.150000  27500.000000   
Microlensing                       1      1   2    1825.000000   3300.000000   
Orbital Brightness Modulation      1      2   2       0.240104      0.342887   
Pulsar Timing                      1      3   3       0.090706     66.541900   
Pulsation Timing Variations        1      1   1    1170.000000   1170.000000   
Radial Velocity                    1      1   6       0.736540    360.200000   
Transit                            1      1   7       0.355000      5.714932   
Transit Timing Variations          2      2   3      22.339500     57.011000   

                                                mass               distance  \
                                         max     min median    max      min   
method                                                                        
Astrometry                       1016.000000     NaN    NaN    NaN    14.98   
Eclipse Timing Variations       10220.000000  4.2000  5.125   6.05   130.72   
Imaging                        730000.000000     NaN    NaN    NaN     7.69   
Microlensing                     5100.000000     NaN    NaN    NaN  1760.00   
Orbital Brightness Modulation       1.544929     NaN    NaN    NaN  1180.00   
Pulsar Timing                   36525.000000     NaN    NaN    NaN  1200.00   
Pulsation Timing Variations      1170.000000     NaN    NaN    NaN      NaN   
Radial Velocity                 17337.500000  0.0036  1.260  25.00     1.35   
Transit                           331.600590  1.4700  1.470   1.47    38.00   
Transit Timing Variations         160.000000     NaN    NaN    NaN   339.00   

                                                  year                
                                 median      max   min  median   max  
method                                                                
Astrometry                       17.875    20.77  2010  2011.5  2013  
Eclipse Timing Variations       315.360   500.00  2008  2010.0  2012  
Imaging                          40.395   165.00  2004  2009.0  2013  
Microlensing                   3840.000  7720.00  2004  2010.0  2013  
Orbital Brightness Modulation  1180.000  1180.00  2011  2011.0  2013  
Pulsar Timing                  1200.000  1200.00  1992  1994.0  2011  
Pulsation Timing Variations         NaN      NaN  2007  2007.0  2007  
Radial Velocity                  40.445   354.00  1989  2009.0  2014  
Transit                         341.000  8500.00  2002  2012.0  2014  
Transit Timing Variations       855.000  2119.00  2011  2012.5  2014

### ピボットテーブルによる多次元集計
groupbyの多次元版

In [64]:
import numpy as np
import pandas as pd
import seaborn as sns
titanic = sns.load_dataset('titanic')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
